<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/4_Teste_Massivo_Modularizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nomes = [] # Armazena o nome do frame que está sendo analisado

formas = [] # Armazena as formas de cada objeto

areas = [] # Armazena a área de cada objeto
perimeters = [] # Armazena o perímetro de cada objeto

width = [] # Armazena a largura do retângulo de contorno de cada objeto
height = [] # Armazena a altura do retângulo de contorno de cada objeto
angle = [] # Armazena o ângulo (em graus) do retângulo de contorno de cada objeto
radius = [] # Armazena o raio do círculo de contorno de cada objeto

equivalent_diameter = [] # Armazena o diâmetro equivalente (em pixels) do círculo com a mesma área de cada objeto

centroids_x = [] # Armazena as coordenadas x do centróide de cada objeto
centroids_y = [] # Armazena as coordenadas y do centróide de cada objeto
aspect_ratio = [] # Armazena a relação entre largura e altura do retângulo de contorno de cada objeto
extent = [] # Armazena a razão entre a área do contorno e a área do retângulo de contorno de cada objeto
solidity = [] # Armazena a razão entre a área do contorno e a área do casco convexo de cada objeto

In [ ]:
def get_frame_name(frame):
  # Divide o caminho do arquivo e obtém o nome do frame
  nome_completo = frame.split('/')
  nome = nome_completo[int(posicao_do_nome_frame)].split('.')
  return nome[0]

In [ ]:
def print_frame_info(nome):
  # Imprime informações do frame
  print("================================================================================")
  print("================================================================================")
  print(nome)

In [ ]:
def apply_filters(image):
  # Converte a imagem para float32
  image32f = np.float32(image)
  
  # Aplica um filtro de média
  mu = cv2.blur(image32f, (3, 3))

  # Calcula a variância da imagem filtrada
  mu2 = cv2.blur(image32f * image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu * mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento Gaussiano
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

  # Aplica o filtro Laplaciano
  laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

  # Converte a saída para o intervalo de 0 a 255
  sigma = cv2.convertScaleAbs(laplacian)

  return sigma

In [ ]:
def apply_threshold(image):
  # Calcula o gradiente da imagem
  gradient_magnitude = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)

  # Normaliza a magnitude do gradiente
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Aplica uma limiarização utilizando o método de Otsu
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  
  return gradient_threshold

In [ ]:
def dilate_image(image):
  # Define o kernel para a dilatação
  kernel = np.ones((5, 5), np.uint8)

  # Dilata a imagem
  image = cv2.dilate(image, kernel, iterations=2)
  return image

In [ ]:
def find_contours(image):
  # Encontra os contornos na imagem
  contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  return contours

In [ ]:
def calculate_contour_properties(cnt):
    # Calcula a área do contorno
    area = cv2.contourArea(cnt)

    # Calcula o perímetro do contorno
    perimeter = cv2.arcLength(cnt, True)
    
    # Calcula a circularidade do contorno
    if perimeter > 0:
        circularity = 4 * np.pi * area / (perimeter ** 2)
    else:
        circularity = 0
    return area, perimeter, circularity

In [ ]:
def process_contours(contours, imagem_original):

  objetos = 0

  for cnt in contours:
    # Aproxima o contorno por uma sequência de segmentos de linha
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula as propriedades do contorno
    area, perimeter, circularity = calculate_contour_properties(cnt)

    if circularity > 0.30:
      forma = "circulo"

      # Obtém o raio do círculo
      radius = get_circle_radius(cnt)

      # Verifica se é um círculo válido
      if is_valid_circle(area, radius):
        # Desenha o círculo na imagem
        draw_circle(imagem_original, cnt)
        objetos += 1

      else:
        forma = "quadrado"

        # Verifica se é um quadrado válido
        if is_valid_square(area, approx):
          # Desenha o quadrado na imagem
          draw_square(imagem_original, approx)
          objetos += 1

  return imagem_original, objetos

In [ ]:
def process_frame(frame):
  # Carrega a imagem
  im = cv2.imread(frame)
  
  # Obtém o nome do frame
  nome = get_frame_name(frame)

  # Imprime informações do frame
  print_frame_info(nome)

  # Cria uma cópia da imagem para desenhar os contornos
  img_contours = im.copy()

  # Converte a imagem para escala de cinza
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Aplica uma série de filtros à imagem
  sigma = apply_filters(image)

  # Aplica uma limiarização à imagem filtrada
  gradient_threshold = apply_threshold(sigma)

  # Dilata a imagem limiarizada
  gradient_threshold = dilate_image(gradient_threshold)

  # Encontra os contornos na imagem dilatada
  contours, hierarchy = find_contours(gradient_threshold)

  # Processa cada contorno encontrado
  objetos = process_contours(contours, img_contours)

  # Imprime o número de objetos encontrados
  print_object_count(objetos)

  # Exibe a imagem com os contornos encontrados
  show_image(img_contours, nome, objetos)

In [ ]:
def extract_frame_from_video(video, destino, cont=1):
  # Verificar se o caminho do vídeo existe
  if not os.path.exists(video):
    print(f"O vídeo '{video}' não existe.")
    return

  # Abrir o vídeo
  vidcap = cv2.VideoCapture(video)

  # Verificar se o vídeo foi aberto corretamente
  if not vidcap.isOpened():
    print(f"Falha ao abrir o vídeo '{video}'.")
    return

  interval = 60  # intervalo em segundos (1 minuto)

  # Pular frames com base no intervalo definido
  vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(interval * cont * vidcap.get(cv2.CAP_PROP_FPS)))

  # Extrair o nome do vídeo a partir do caminho
  nome_completo = video.split('/')
  nome = nome_completo[int(posicao_do_nome_video)].split('.')
  nome = nome[0]

  # Ler o próximo frame
  success, image = vidcap.read()

  # Verificar se o frame foi lido com sucesso
  if not success:
    print(f"Falha ao ler a imagem {cont} do vídeo '{video}'.")

    # Liberar os recursos utilizados pelo objeto de leitura do vídeo
    vidcap.release()

    return

  # Salvar o frame lido
  cv2.imwrite("{}{}.jpg".format(destino, nome), image)
  print(f"Frame {cont} extraído do vídeo '{nome}' salva com sucesso em '{destino}'.")

  # Liberar os recursos utilizados pelo objeto de leitura do vídeo
  vidcap.release()
  # Fechar todas as janelas abertas pelo OpenCV
  cv2.destroyAllWindows()

In [ ]:
def extract_frames_times(path_list, destino):
  for video in path_list:
    # Verificar se o caminho do vídeo existe
    if not os.path.exists(video):
      print(f"O vídeo '{video}' não existe.")
      return

    # Abrir o arquivo de vídeo
    vidcap = cv2.VideoCapture(video)

    # Verificar se o vídeo foi aberto corretamente
    if not vidcap.isOpened():
      print(f"Falha ao abrir o vídeo '{video}'.")
      return

    # Extrair o nome do vídeo a partir do caminho
    nome_completo = video.split('/')
    nome = nome_completo[int(posicao_do_nome_video)].split('.')
    nome = nome[0]

    # create a folder by the name of the video file
    if not os.path.exists("{}".format(destino)):
      os.makedirs("{}".format(destino))

    # Ler o primeiro frame
    success, image = vidcap.read()
    print('Read a new frame: ', success)

    # Verificar se houve sucesso na leitura do frame
    if not success:
      print(f"Falha ao ler o frame.")

      # Liberar os recursos utilizados pelo objeto de leitura do vídeo
      vidcap.release()
    
      break

    # Salvar o frame lido
    cv2.imwrite("{}{}.jpg".format(destino, nome), image)
    print('Creating...' + "{}{}.jpg".format(destino, nome))

    # Liberar os recursos utilizados pelo objeto de leitura do vídeo
    vidcap.release()
    # Fechar todas as janelas abertas pelo OpenCV
    cv2.destroyAllWindows()

In [ ]:
# entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 15/Vídeos separados - TR e TT/grey_field_videos/*.mp4')

entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/*.mpg')

# Imprime os elementos da lista com quebras de linha entre cada elemento
print('\n'.join(entrada))

In [ ]:
posicao_do_nome_video = input("Posição do nome do video: ") #9

In [ ]:
destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_grey/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/'

In [ ]:
frames = glob.glob(destino + '*.jpg')
print('\n'.join(frames))

In [ ]:
posicao_do_nome_frame = input("posicao_do_nome_frame:") #6

In [ ]:
# extract_frames_from_videos(entrada, destino)

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in frames:
  process_frame(frame)